In [2]:
from pathlib import Path

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from collections import OrderedDict

In [3]:
data_path = Path('data')

!ls $data_path

stability_diversity_full.csv	stability.h5
stability_diversity_test.csv	stability_test.csv
stability_diversity_train.csv	stability_test.h5
stability_embeddings_test.pkl	stability_train.csv
stability_embeddings_train.pkl	stability_train.h5
stability_embeddings_val.pkl


In [4]:
def open_sets(base_path):
    sets = {}

    for path in base_path.glob('*.csv'):
        fname = path.stem
        kind = fname.split('_')[1]

        df = pd.read_csv(path)
        sets[kind] = df
        
    return sets

sets = open_sets(data_path)
sets['train']

,sequence,consensus_stability_score
0,GSSQETIEVEDEEEARRVAKELRKKGYEVKDERRGNKWHVHRT,0.37
1,TLDEARELVERAKKEGTGMDVNGQRFEDWREAERWVREQEKNK,0.62
2,TELKKKLEEALKKGEEVRVKFNGIEIRNTSEDAARKAVELLEK,-0.03
3,GSSQETIEVEDEEEARRVAKELRKTGYEVKIERRGNKWHVHRT,1.41
4,TTIHVGDLTLKYDNPKKAYEIAKKLAKKYNLQVTIKNGKITVT,1.11
...,...,...
7705,GSSKTQYEYDTKEEHQKAYEKFKKQGIPVTITQKNGKWFVQVE,0.80
7706,TIDEIIKALEQAVKDNKPIQVGNYTVTSADEAEKLAKKLKKPY,0.82
7707,TQDEIIKALEQAVKDNKPIQVGNYTVTSADEAEKLAKKLKKEY,0.66
7708,TTIKVNGQEYTVPLSPEQAAKAAKKRWPDYEVQIHGNTVWVTR,1.05


## Data Duplication

In [5]:
len(sets['train']['sequence'].unique()) != len(sets['train'])

False

## Alignments

In [6]:
from Bio import pairwise2

alignment = pairwise2.align.globalxx(sets['train']['sequence'][0], sets['train']['sequence'][1])
print(pairwise2.format_alignment(*alignment[0]))

ModuleNotFoundError: No module named 'Bio'

## Score per metrics

* More unique values => more stability?
* Does sequence lenght matter?


In [ ]:
def append_to_key(d, k, v):
    if k in d:
        values = d[k]
        values.append(v)
        d[k] = values
        
    else:
        d[k] = [v]
        
def compute_mean_scores(d):
    new_d = dict()
    for k in d:
        mean_score = np.mean(d[k])
        new_d[k] = (mean_score, len(d[k]))
        
    return  OrderedDict(sorted(new_d.items()))

In [ ]:
score_per_length = dict()
score_per_uniqueness = dict()

for idx, (seq, score) in sets['train'].iterrows():
    # score_per_length
    size = len(seq)
    append_to_key(score_per_length, size, score)
    
    
    # score_per_uniqueness
    uniques = set(seq)
    uniq_size = len(uniques)
    
    append_to_key(score_per_uniqueness, uniq_size, score)
    
    # count unique aminoacids
    for amino in uniques:
        uniques.add(amino)


In [ ]:
compute_mean_scores(score_per_length)

In [ ]:
compute_mean_scores(score_per_uniqueness)

## By Score

In [ ]:
sorted_train = sets['train'].sort_values(by='consensus_stability_score').reset_index()

In [ ]:
alignment = pairwise2.align.globalxx(sorted_train['sequence'][0], sorted_train['sequence'][1])
print(pairwise2.format_alignment(*alignment[0]))

sorted_train.head(20)

In [ ]:
alignment = pairwise2.align.globalxx(
    sorted_train['sequence'][len(sorted_train)-1], 
    sorted_train['sequence'][len(sorted_train)-2]
)

print(pairwise2.format_alignment(*alignment[0]))

sorted_train.tail(20)

* Diversity Maximizing: This is a greedy strategy which starts from the reference and adds the sequence with highest average hamming distance to current set of sequences.

* Diversity Minimizing: This strategy is equivalent to the Diversity Maximizing strategy, but adds the se- quence with lowest average hamming distance. It is used to explore the effects of diversity on model per- formance.

* HHFilter: This strategy applies hhfilter (Steinegger etal.,2019)withthe-diff Mparameter,whichre- turns M or more sequences that maximize diversity (the result is usually close to M). If more than M sequences are returned we apply the Diversity Maxi- mizing strategy on top of the output.

In [ ]:
alignment[0].score

In [68]:
from scipy.spatial import distance

import random
import numpy as np
from numba import jit

from tqdm.autonotebook import tqdm

def compute_diversity(seq1, seq2, method = "hamming"):
    if method == "alignment":
        method = pairwise2.align.globalxx
        diversity = method(seq1, seq2)
        diversity = diversity[0].score / max(len(seq1), len(seq2))
        
    elif method == "hamming":
        if len(seq1) != len(seq2):
            return np.nan
        
        method = distance.hamming
        diversity = method(list(seq1), list(seq2))
        
    return diversity


def dataset_diversity(sequences, method = "hamming", reduce = "mean", verbose = True):
    if reduce != "mean":
        raise NotImplementedError
    else:
        # nan results are due to different lengths
        reducer = np.nanmean
    
    reduced_divs = []
    if verbose: 
        pbar = tqdm(total = len(sequences), miniters = 1, smoothing = 1)

    for seq_idx, seq in enumerate(sequences):
        other_sequences = np.concatenate((sequences[:seq_idx] , sequences[seq_idx + 1:]))
        
        if seq in other_sequences:
            raise IndexError(f"{seq_idx} is in other_sequences, e.g. it is not correctly indexed")
        

        v_diversity = np.vectorize(lambda x: compute_diversity(seq, x))
        if len(other_sequences) > 1:
            div_vs_all = v_diversity(other_sequences)  
        else:
            print(f"Skipping sequence {seq_idx}")
            continue
        
        
        reduced_div_vs_all = reducer(div_vs_all) if len(div_vs_all) >= 1 else np.nan
        reduced_divs.append(reduced_div_vs_all)
        
        if verbose:
            pbar.update(1)
            pbar.refresh()
    
    if verbose:
        pbar.close()
        
    return reduced_divs

def sample_by_diversity(sequences, size, min_score = 0.7, args = {"verbose" : False}):
    check_diversity = lambda ds : np.nanmean(dataset_diversity(np.array(list(ds)), **args)) >= min_score
    
    sampled_seqs = set()
    max_iters = 10
    current_iters = 0
    while len(sampled_seqs) < size:
        idx_sample = random.sample(range(len(sequences)), 1)
        sample = sequences[idx_sample]
        
        is_empty = len(sampled_seqs) == 0 or len(sampled_seqs) == 1
        
        if is_empty or check_diversity(sampled_seqs.union({sample[0]})):
            sampled_seqs.add(sample[0])
            sequences = np.delete(sequences, idx_sample)
            if len(sampled_seqs) % (size // 10) == 0:
                print(len(sampled_seqs))
            
        else: 
            continue
    # check diversity
    if not check_diversity(sampled_seqs):
        raise ValueError("Sampled diversity error")
        
    return sampled_seqs

def select_by_diversity(sequences, diversities, min_size, min_score = 0.7, args = {"verbose" : False}):
    chosen_seqs = []
    
    min_diversity = lambda ds : np.nanmean(dataset_diversity(np.array(list(ds)), **args)) >= min_score
    diminishes_diversity = lambda idx: diversities[idx] > min_size
    around_size = lambda seqs: len(seqs) * 1.2 >= min_size
    is_empty = len(chosen_seqs) == 0 or len(chosen_seqs) == 1
    

    with tqdm(total = min_size, miniters = 10, smoothing = 1) as pbar:
        last_diversity = False
        while len(chosen_seqs) <= min_size:
            max_iters = 10
            current_iters = 0

            seq_set = set(chosen_seqs).union({sequences[0]}) 
        
            if is_empty or not diminishes_diversity(0) or min_diversity(seq_set):
                chosen_seqs.append(sequences[0])
                sequences = np.delete(sequences, 0)
                diversities = np.delete(diversities, 0)

                current_iters = 0
            pbar.update(1)

            if current_iters >= max_iters:
                break
            else:
                current_iters += 1

            if diminishes_diversity(0) and min_diversity(seq_set) and around_size(chosen_seqs):
                break
            
    return chosen_seqs

In [69]:
diversities_train = dataset_diversity(sets["train"]["sequence"].values)

  0%|          | 0/7710 [00:00<?, ?it/s]

In [83]:
diversities_train[:10]

[0.8394134005304014,
 0.8469666259574454,
 0.8795092981365058,
 0.83650682658194,
 0.8295142928034904,
 0.8158611289236332,
 0.836996626183011,
 0.043124059151686885,
 0.8282833490692195,
 0.87912583660672]

In [84]:
pd.read_csv("data/stability_diversity_train.csv", index_col=0)[:10]

,sequence,consensus_stability_score,diversity
0,GSSQETIEVEDEEEARRVAKELRKKGYEVKDERRGNKWHVHRT,0.37,0.839413
1,TLDEARELVERAKKEGTGMDVNGQRFEDWREAERWVREQEKNK,0.62,0.846967
2,TELKKKLEEALKKGEEVRVKFNGIEIRNTSEDAARKAVELLEK,-0.03,0.879509
3,GSSQETIEVEDEEEARRVAKELRKTGYEVKIERRGNKWHVHRT,1.41,0.836507
4,TTIHVGDLTLKYDNPKKAYEIAKKLAKKYNLQVTIKNGKITVT,1.11,0.829514
5,SKDEAQREAERAIRSGNKEEARRILEEVGYSPEQAERIIRKLG,1.24,0.815861
6,TIDEIIKALEQAVKDGKPIQVGNYTVTSADEAEKLAKKLKKEY,1.05,0.836997
7,FEIPDDVPLPAGWEMARTSSGQRYFKNHIDQTTTWQDPRKAMLSQM,0.89,0.043124
8,TTIHVGDLTLKYDNPKKAYEIAKKLDKKYNLTVTIKNGKITVT,0.88,0.828283
9,GSSGSLSDEDFKAVFGMTRSAFAMLPLWKQQNLKKEKGLFGSS,1.15,0.879126


In [48]:
# ds_div_train = dataset_diversity(sets["train"]["sequence"])
# sets["train"]["diversity"] = np.nanmean(np.array(ds_div_train), 1)
# np.nanmean(np.array(ds_div_train))
# 0.8xxx

In [49]:
# ds_div_test = dataset_diversity(sets["test"]["sequence"])
# sets["test"]["diversity"] = np.nanmean(np.array(ds_div_test), 1)
# np.nanmean(np.array(ds_div_test))

In [50]:
# sets["train"].to_csv(f"{data_path}/stability_diversity_train.csv")
# sets["test"].to_csv(f"{data_path}/stability_diversity_test.csv")

In [51]:
sets = {
    'train' : pd.read_csv(f"{data_path}/stability_diversity_train.csv"),
    'test'  : pd.read_csv(f"{data_path}/stability_diversity_test.csv") 
}

sorted_div = {
    "train" : sets["train"].sort_values(by="diversity", axis=0, ascending=False),
    "test"  : sets["test"].sort_values(by="diversity", axis=0, ascending=False),
}

In [226]:
import torch
from torch.utils.data import Dataset

class ProteinStabilityDataset(Dataset):
    """Protein1D Stability dataset."""

    def __init__(self, proteins_path, ret_dict = True):
        """
        Args:
            proteins_path (string): Path to the H5Py file that contains sequences and embeddings.
        """
        self.stability_path = proteins_path
        self.ret_dict = ret_dict

    def __len__(self):
        return len(h5py.File(self.stability_path, "r")['sequences'])
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        with h5py.File(self.stability_path, "r") as dset:
            sequences = dset['sequences'][idx]
            embeddings = dset['embeddings'][idx]
            labels = dset['labels'][idx]
            
            sample = {
                'sequences': sequences, 
                'embeddings': torch.from_numpy(embeddings), 
                'labels': torch.Tensor([labels])
            }

        return sample if self.ret_dict else (sample['embeddings'], sample['labels'])

In [227]:
from torch.utils.data import Sampler, DataLoader, SubsetRandomSampler
from typing import List, Sized, Iterator

import numpy as np

class SubsetDiversitySampler(Sampler):
    """Samples elements given their diversity w.r.t. the rest of the dataset"""
    
    def __init__(self, dataset: Sized, diversity_path : str, diversity_cutoff : float, strategy : str = "maximize", seed : int = 123) -> None:
        """
        Args:
          diversity_path (string): Path to the csv with sequences and diversity.
        """
        self.diversity_path = diversity_path
        self.diversity_data = pd.read_csv(self.diversity_path, index_col=0)
        
        if strategy == "maximize":
            sorting_order = False
        elif strategy == "minimize":
            sorting_order = True
        else: 
            raise ValueError(f"Strategy {strategy} is not supported")
        
        self.diversity_data = self.diversity_data.sort_values(
            by='diversity', 
            ascending=sorting_order
        )
        
        self.cutoff = diversity_cutoff
        self.indices = []
        self.strategy = strategy
        self.seed = seed
        
        if strategy == 'maximize':
            self.cutoff_lambda = lambda x, cutoff : x < cutoff
        elif strategy == 'minimize':
            self.cutoff_lambda = lambda x, cutoff : x > cutoff
            
        self.subset_by_cutoff(self.cutoff, self.cutoff_lambda)
            
    def subset_by_cutoff(self, cutoff, cutoff_func) -> None:
        for row in self.diversity_data.itertuples():
            is_cutoff = cutoff_func(row.diversity, cutoff)
            if is_cutoff:
                break
            else:
                self.indices.append(row.Index)
        
    def __iter__(self) -> Iterator[int]:
        rng = np.random.default_rng(self.seed)
        return iter(rng.permutation(self.indices))

    def __len__(self) -> int:
        return len(self.indices)

In [238]:
dataset = ProteinStabilityDataset("data/stability.h5", ret_dict=True)
sampler = SubsetDiversitySampler(dataset, "data/stability_diversity_full.csv", 0.85)
dl = DataLoader(dataset, batch_size=128, sampler=sampler)

# check diversity sampler
df = pd.read_csv("data/stability_diversity_full.csv", index_col=0)
for batch in dl:
    for seq in batch["sequences"]:
        row = df[df.sequence == str(seq, 'utf-8')]
        if float(row.diversity.values) < 0.85:
            raise AssertionError

In [218]:
# for s in [1000, 1500, 2000, 3000, 4000, 5000, 6000]:
#     for kind in ["train"]:
#         chosen_by_diversity = select_by_diversity(
#             sequences=sorted_div[kind]["sequence"].values, 
#             diversities=sorted_div[kind]["diversity"].values,
#             min_size=s, 
#             min_score=0.78
#         )
    
#     subset = sets["train"][sets["train"]["sequence"].isin(chosen_by_diversity)]
#     subset.to_csv(f"{data_path}/stability_diversity_train_{s}.csv", index=False)